**Домашнее задание к лекции "Основы веб-скрапинга и работы с API"**

**Задание 1.**
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
import time

In [4]:
import regex

In [5]:
import pandas as pd

In [6]:
keywords = ['Python', 'python', 'Парсинг','парсинг']

In [7]:
URL = 'http://habr.com/ru/all/'

In [8]:
def get_all_info(url, keywords, pages):
    habr_posts = pd.DataFrame()
    
    for i in range(pages):
        params = 'page' + str(i + 1)
        res = requests.get(url+params)
        time.sleep(0.3)
        soup = BeautifulSoup(res.text, 'html.parser')
        title_blocks = soup.find_all('h2', class_='post__title')
        text_blocks = soup.find_all('div', class_='post__text post__text-html post__text_v1')
        time_block = soup.find_all('span', class_='post__time')
        all_titles = list(map(lambda x: x.find('a', class_='post__title_link').text, title_blocks))
        all_linkes = list(map(lambda x: x.find('a').get('href'), title_blocks))
        all_text = list(map(lambda x: x.get_text(),text_blocks))
        all_time = list(map(lambda x: x.get_text(), time_block))
        all_info = [list(tup) for tup in zip(all_time, all_titles, all_text, all_linkes)]
        for j in range(len(all_info)):
            if regex.search(r"\L<words>", all_info[j][1], words=keywords) or\
            regex.search(r"\L<words>", all_info[j][2], words=keywords):
                row = {'time': all_info[j][0], 'title': all_info[j][1], 'text': all_info[j][2], 'link': all_info[j][3]}
                habr_posts = pd.concat([habr_posts, pd.DataFrame([row])]) 
                 
    return habr_posts

all_links = get_all_info(URL, keywords, 30)
all_links

,time,title,text,link
0,сегодня в 11:14,Моделирование звука гитарных нот с помощью алг...,"\n Знакомьтесь, эталонная нота ля первой ...",https://habr.com/ru/post/514844/
0,сегодня в 10:58,«Я что-то накодил и все упало»: провалы в Pyth...,\n На конференциях обычно принято рассказ...,https://habr.com/ru/company/oleg-bunin/blog/51...
0,10 августа 2020 в 16:16,"А вы знаете о том, что в Python есть встроенна...","\n Если вы — программист, то я полагаю, ч...",https://habr.com/ru/company/ruvds/blog/514538/
0,9 августа 2020 в 21:40,"Три редко используемых возможности Python 3, о...",\n\r\nPython 3 существует уже какое-то время и...,https://habr.com/ru/post/514444/
0,9 августа 2020 в 08:21,Мониторинг демон на Asyncio + Dependency Injec...,"\n Привет,\n\r\nЯ создатель Dependency In...",https://habr.com/ru/post/514384/
0,8 августа 2020 в 15:02,"Шесть Python-пакетов, которые рекомендуется ис...",\n\nЕсть бесчисленное множество Python-пакетов...,https://habr.com/ru/post/514348/
0,8 августа 2020 в 14:24,Реализация offline режима для Yandex.Music,\nВведение\nСегодня мы будем рассматривать так...,https://habr.com/ru/post/514342/
0,7 августа 2020 в 13:53,Рецепты uWSGI: преобразование документов с исп...,\n Для приготовления преобразования докум...,https://habr.com/ru/post/514220/
0,7 августа 2020 в 11:27,TensorFlow.js: Часть 1: Использование Low-Leve...,\n В настоящее время Python занимает доми...,https://habr.com/ru/post/513172/
0,6 августа 2020 в 18:08,Как работают профайлеры в Ruby и Python?,\nПеревод статьи подготовлен в преддверии стар...,https://habr.com/ru/company/otus/blog/514132/


**Задание 2.**
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [9]:
import json

In [10]:
URL = 'https://www.avast.com/hackcheck'

In [11]:
EMAIL = ['golobova77@mail.ru',  'kovalev@mail.ru', 'nt-centr77@yandex.ru']

In [12]:
leak_inform = pd.DataFrame()
for i in range(len(EMAIL)):
    URL = 'https://digibody.avast.com/v1/web/leaks'
    body = {}
    body['email'] = EMAIL[i]
    req = requests.post(URL, data=json.dumps(body))
    result = json.loads(req.text)
    for j in range(len(result['value'])):
        row = {'email': body['email'],\
           'data': result['value'][j]['leak_info']['date'],\
           'title': result['value'][j]['leak_info']['title'],\
           'description': result['value'][j]['leak_info']['description']}
        leak_inform = pd.concat([leak_inform, pd.DataFrame([row])]) 
leak_inform
    

,email,data,title,description
0,golobova77@mail.ru,1513900800000,Combolist of 1.4 Billion Credentials,The proliferation of stolen or leaked database...
0,golobova77@mail.ru,1549411200000,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix..."
0,golobova77@mail.ru,1549411200000,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix..."
0,golobova77@mail.ru,1477612800000,Badoo,"In June 2016, millions of accounts compromised..."
0,golobova77@mail.ru,1548720000000,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix..."
0,golobova77@mail.ru,1538611200000,AutoWebinar FM,"In December 2013, AutoWebinar FM's database wa..."
0,golobova77@mail.ru,1549411200000,AP MYR and Zabugor Combo List,"On January 7, 2019, an online user named Sanix..."
0,golobova77@mail.ru,1516406400000,K-Vrachu,"In January 2016, K-Vrachu's database was alleg..."
0,kovalev@mail.ru,1549411200000,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix..."
0,kovalev@mail.ru,1575504000000,Sensitive Source,This source has been marked as sensitive due t...


In [13]:
# Request URL: https://digibody.avast.com/v1/web/email/subscribe

URL = 'https://digibody.avast.com/v1/web/email/subscribe'
body = {'email': 'golobova77@mail.ru'}
req = requests.post(URL, data=json.dumps(body))
print(req.status_code)
print(req.text)

200
{"status":"ok","value":{"already_subscribed":true}}


In [14]:
# Request URL: https://digibody.avast.com/v1/web/email/subscribe

URL = 'https://digibody.avast.com/v1/web/leaks'
body = {'email': 'golobova77@mail.ru'}
req = requests.post(URL, data=json.dumps(body))
print(req.status_code)
print(req.text)

200
{"status":"ok","value":[{"leak_id":"037a277e-d484-4c34-8473-904024e489e1","username":"golobova77@mail.ru","domain":"","passwords":[{"password":null,"encrypted":false}],"leak_info":{"id":"037a277e-d484-4c34-8473-904024e489e1","date":1513900800000,"title":"Combolist of 1.4 Billion Credentials","description":"The proliferation of stolen or leaked databases has given rise to credential stuffing, a fairly simple technique in which criminals load lists of stolen credentials, called combo lists, into automated brute-forcing tools to test credentials en masse. These tools test stolen passwords against thousands of targeted websites and applications until there is a match.\n\nThis particular combo list was likely compiled over time from a variety of public and private breaches. It contains approximately 1.4 billion email and password records. Criminals are actively leveraging this list, along with credential stuffing tools, to gain unauthorized access to targeted websites. This list has bee